# Лабораторная работа 3
> K33401 - Рейнгеверц Вадим

[Ссылка на Google Colab](https://colab.research.google.com/drive/1ccoOx3daKB7TbsYaaSi8tfiw8MeIL9j8)
## Анализ эмоциональной окраски пользовательских постов

На практике анализ текстовых данных в социальных сетях может быть полезен для целого ряда задач. К примеру, можно распознавать токсичные посты для их модерации или анализировать тональность текста, чтобы оценить отношение группы людей к тому или иному явлению. В этой лабораторной работе мы будем исследовать пользовательские посты, посвящённые COVID-19

## 1.&nbsp;Скачайте набор данных Covid 19 Indian Sentiments on covid19 and lockdown по ссылке

[Covid 19 Indian Sentiments on covid19 and lockdown](https://www.kaggle.com/datasets/surajkum1198/twitterdata)

С его помощью мы будем обучать модель распознавать настроение автора поста. 

In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "whydoihavetodothiseh"
os.environ['KAGGLE_KEY'] = "1c0534cf33c5d00bb380dc4aebadd907"

!kaggle datasets download -d surajkum1198/twitterdata
!unzip -o $(ls -hat | grep -m 1 ".zip")

print()
!ls | grep ".csv"

twitterdata.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  covid19-tweets.zip
  inflating: covid19_tweets.csv      

covid19_tweets.csv
finalSentimentdata2.csv


In [2]:
import numpy as np
import pandas as pd

sentiments = pd.read_csv("finalSentimentdata2.csv")
sentiments.head()

,Unnamed: 0,sentiment,text
0,3204,sad,agree the poor in india are treated badly thei...
1,1431,joy,if only i could have spent the with this cutie...
2,654,joy,will nature conservation remain a priority in ...
3,2530,sad,coronavirus disappearing in italy show this to...
4,2296,sad,uk records lowest daily virus death toll since...


## 2.&nbsp;В датафрейме оставьте только те строки, у которых в столбце sentiment написано sad и joy

Для простоты выполнения оставляем только 2 возможных настроения, строки с постами, не соответствующими им, можно удалить.

In [3]:
print("Sentiments:", sentiments['sentiment'].unique(), sentiments.shape)

sentiments = sentiments[sentiments["sentiment"].isin(["sad", "joy"])]

print("Sentiments:", sentiments['sentiment'].unique(), sentiments.shape)

Sentiments: ['sad' 'joy' 'fear' 'anger'] (3090, 3)
Sentiments: ['sad' 'joy'] (1522, 3)


## 3.&nbsp;Примените на этом наборе данных шаги предварительной обработки

К шагам предварительной обработки относятся:

1. приведение к нижнему регистру,
2. токенизация,
3. удаление стоп-слов (обратите внимание на то, что стоп-слова должны соответствовать языку, с которым ведётся работа, в данном случае это английский),
4. уберите заведомо неинформативные данные (хэштеги и отметки пользователей).

Также необходимо произвести векторизацию.

Пример выполнения этих шагов на другом наборе можно подсмотреть здесь:

[Google Colaboratory](https://colab.research.google.com/drive/14dcDHynxOyhb510HO2lmsoFmBSLO5Vfx?usp=sharing)

Чтобы улучшить качество обучения можно попробовать перед векторизацией также применить лемматизацию (приведение слов к исходной форме). Для этого подойдёт, к примеру, nltk.WordNetLemmatizer (pymorphy2).

[Плавное введение в Natural Language Processing (NLP)](https://datastart.ru/blog/read/plavnoe-vvedenie-v-natural-language-processing-nlp)

In [4]:
def to_lower(string):
    return string.lower() if isinstance(string, str) else string

lowercased_text = sentiments.apply(lambda row: to_lower(row['text']), axis=1)
lowercased_text.head()

0    agree the poor in india are treated badly thei...
1    if only i could have spent the with this cutie...
2    will nature conservation remain a priority in ...
3    coronavirus disappearing in italy show this to...
4    uk records lowest daily virus death toll since...
dtype: object

In [5]:
import re

def filter_out_meta(text):
    # filter out #hashtags and @mentions
    # https://stackoverflow.com/a/8377440
    return ' '.join(re.sub("(#[A-Za-z0-9]+)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split())

filtered_text = lowercased_text.apply(lambda row: filter_out_meta(row))
filtered_text.head()

0    agree the poor in india are treated badly thei...
1    if only i could have spent the with this cutie...
2    will nature conservation remain a priority in ...
3    coronavirus disappearing in italy show this to...
4    uk records lowest daily virus death toll since...
dtype: object

In [6]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

tokens = filtered_text.apply(lambda row: word_tokenize(row))
tokens.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0    [agree, the, poor, in, india, are, treated, ba...
1    [if, only, i, could, have, spent, the, with, t...
2    [will, nature, conservation, remain, a, priori...
3    [coronavirus, disappearing, in, italy, show, t...
4    [uk, records, lowest, daily, virus, death, tol...
dtype: object

In [7]:
from nltk.corpus import stopwords
import string 

stop_words = set(stopwords.words("english"))
punctuation = set(string.punctuation)
punctuation = punctuation.union(set(["``", "..", "..."]))

cleaned_tokens = tokens.apply(lambda row: [token for token in row if token not in stop_words and token not in punctuation])
cleaned_tokens.head()

0    [agree, poor, india, treated, badly, poors, se...
1    [could, spent, cutie, vc, sakshi, n, g, h, coa...
2    [nature, conservation, remain, priority, post,...
3    [coronavirus, disappearing, italy, show, intel...
4    [uk, records, lowest, daily, virus, death, tol...
dtype: object

In [8]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

nltk.download('wordnet')
nltk.download('omw-1.4')

# print(lemmatizer.lemmatize("rocks")) # rocks -> rock
lemmatized_tokens = cleaned_tokens.apply(lambda row: list(map(lemmatizer.lemmatize, row)))
lemmatized_tokens.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


0    [agree, poor, india, treated, badly, poor, see...
1    [could, spent, cutie, vc, sakshi, n, g, h, coa...
2    [nature, conservation, remain, priority, post,...
3    [coronavirus, disappearing, italy, show, intel...
4    [uk, record, lowest, daily, virus, death, toll...
dtype: object

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer=lambda x: x, max_df=1.0, min_df=1, max_features=None)

vectorized_tokens = vectorizer.fit_transform(lemmatized_tokens)

## 4.&nbsp;Обучите классификатор

Можно использовать реализацию из библиотеки Scikit-learn


```python
#импорт классификатора
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
#обучаем с помощью векторизованной обучающей выборки
clf.fit(vectorized_x_train, y_train)
#импорт метрик качества
from sklearn.metrics import * 
#получаем предсказание для векторизованной тестовой выборки
pred = clf.predict(vectorized_x_test)
#сравниваем реальные метки с предсказанными
print(classification_report(y_test, pred))
```

In [10]:
from sklearn.model_selection import train_test_split

sentiment_points = vectorized_tokens
sentiment_values = sentiments['sentiment']

x_train, x_test, y_train, y_test = train_test_split(sentiment_points, sentiment_values, test_size=0.25, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(1141, 6111) (381, 6111) (1141,) (381,)


In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report


clf = MultinomialNB()
clf.fit(x_train, y_train)

MultinomialNB()

In [12]:
y_pred = clf.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         joy       0.89      0.77      0.83       190
         sad       0.80      0.91      0.85       191

    accuracy                           0.84       381
   macro avg       0.85      0.84      0.84       381
weighted avg       0.85      0.84      0.84       381



## 5.&nbsp;Оцените качество классификации
Обучите еще одну или несколько классических моделей классификаторов и сравните качество. Подберите наиболее оптимальные гиперпараметры векторизатора и классификатора, при которых метрики будут выше. 

### Перебор гиперпараметров TF-IDF Vectorizer'а

#### Вместе с Multinomial Naive Bayes (MNB)

In [13]:
import itertools

def get_ngrams(n, m, step = 1):
    gram_range_list = [list(np.arange(1, n + 1, step)), list(np.arange(1, m + 1, step))]

    grams = list(itertools.product(*gram_range_list))
    ngrams = list(filter(lambda gram: gram[0] <= gram[1], grams))
    return ngrams

textParams = {
    "ngrams": [(2, 2), *get_ngrams(5, 7, 2)]
}
tfidfParams = {
    "max_df": [0.5, 0.75, 1.0],
    "min_df": [1, 0.1, 0.2], 
    "max_features": [None, 1000, 1800, 2500]
}

mnbParams = {
    "alpha": [0.00001, 0.0001, 0.001, 0.1, 1, 10], 
}

svmParams = {
    "C": [1.0, 2.0, 3.0, 4.0],
    "kernel": ['rbf'],
    "degree": [3],
    "gamma": ['scale', 'auto']
}


print(textParams)
print(tfidfParams)
print(mnbParams)
print(svmParams)

{'ngrams': [(2, 2), (1, 1), (1, 3), (1, 5), (1, 7), (3, 3), (3, 5), (3, 7), (5, 5), (5, 7)]}
{'max_df': [0.5, 0.75, 1.0], 'min_df': [1, 0.1, 0.2], 'max_features': [None, 1000, 1800, 2500]}
{'alpha': [1e-05, 0.0001, 0.001, 0.1, 1, 10]}
{'C': [1.0, 2.0, 3.0, 4.0], 'kernel': ['rbf'], 'degree': [3], 'gamma': ['scale', 'auto']}


In [14]:
textParamsKeys = list(map(lambda item: item[0], textParams.items()))
tfidfParamsKeys = list(map(lambda item: item[0], tfidfParams.items()))
mnbParamsKeys = list(map(lambda item: item[0], mnbParams.items()))
svmParamsKeys = list(map(lambda item: item[0], svmParams.items()))

textParamsValues = list(map(lambda item: item[1], textParams.items()))
tfidfParamsValues = list(map(lambda item: item[1], tfidfParams.items()))
mnbParamsValues = list(map(lambda item: item[1], mnbParams.items()))
svmParamsValues = list(map(lambda item: item[1], svmParams.items()))

# Dataframe out of all combinations of the params
mnb_scoring_df = pd.DataFrame(
    list(itertools.product(*textParamsValues, *tfidfParamsValues, *mnbParamsValues)), 
    columns=[*textParamsKeys, *tfidfParamsKeys, *mnbParamsKeys],
    dtype=object # Pandas converts None -> NaN otherwise
)

# mnb_scoring_df.drop(mnb_scoring_df[mnb_scoring_df['min_df'] > mnb_scoring_df['max_df']].index, inplace=True)
mnb_scoring_df["accuracy"] = ""

mnb_scoring_df

,ngrams,max_df,min_df,max_features,alpha,accuracy
0,"(2, 2)",0.5,1,None,0.00001,
1,"(2, 2)",0.5,1,None,0.0001,
2,"(2, 2)",0.5,1,None,0.001,
3,"(2, 2)",0.5,1,None,0.1,
4,"(2, 2)",0.5,1,None,1,
...,...,...,...,...,...,...
2155,"(5, 7)",1.0,0.2,2500,0.0001,
2156,"(5, 7)",1.0,0.2,2500,0.001,
2157,"(5, 7)",1.0,0.2,2500,0.1,
2158,"(5, 7)",1.0,0.2,2500,1,


In [15]:
# for index, param_row in mnb_scoring_df.head(1000).iterrows():
#     if index % round(len(mnb_scoring_df) * 0.01) == 0:
#         progress = f"Progress: {round((index)/len(mnb_scoring_df) * 100)}%"
#         print(progress)

#     mnb_scoring_df.iloc[index]["accuracy"] = score_mnb(mnb_scoring_df, index, param_row)

# print("Progress: 100%")

In [16]:
from google.colab import output
from joblib import Parallel, delayed
from operator import itemgetter
import nltk

def word_ngram(sequence, ngram_range=(1, 1)):
    return list(nltk.everygrams(sequence, min_len=ngram_range[0], max_len=ngram_range[1]))

def beep():
    output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [17]:
%%time
def score_mnb(param_row):
    max_df, min_df, max_features = itemgetter('max_df', 'min_df', 'max_features')(param_row)
    vectorizer = TfidfVectorizer(analyzer=lambda x: x, max_df=max_df, min_df=min_df, max_features=max_features)

    ngrams = itemgetter('ngrams')(param_row)
    ngrammed_tokens = lemmatized_tokens.apply(lambda row: word_ngram(row, ngrams))

    try:
        points = vectorizer.fit_transform(ngrammed_tokens)
        values = sentiments['sentiment']
    except ValueError:
        # Handle bad params
        return 0.0
    
    x_train, x_test, y_train, y_test = train_test_split(points, values, test_size=0.25, random_state=42)

    alpha = itemgetter('alpha')(param_row)
    clf = MultinomialNB(force_alpha=True, alpha=alpha)
    clf.fit(x_train, y_train)

    y_pred = clf.predict(x_test)

    accuracy = classification_report(y_test, y_pred, output_dict=True, zero_division=0)['accuracy']
    return accuracy

print(f"Estimated to take {len(mnb_scoring_df)} fits or {round(len(mnb_scoring_df) / 260)} minutes")

accuracies = Parallel(n_jobs=-1)(delayed(score_mnb)(iterrow[1]) for iterrow in mnb_scoring_df.iterrows())
mnb_scoring_df['accuracy'] = pd.DataFrame(accuracies)
beep()

mnb_scoring_df

Estimated to take 2160 fits or 8 minutes
CPU times: user 26.7 s, sys: 1.07 s, total: 27.8 s
Wall time: 8min 36s


,ngrams,max_df,min_df,max_features,alpha,accuracy
0,"(2, 2)",0.5,1,None,0.00001,0.692913
1,"(2, 2)",0.5,1,None,0.0001,0.692913
2,"(2, 2)",0.5,1,None,0.001,0.653543
3,"(2, 2)",0.5,1,None,0.1,0.643045
4,"(2, 2)",0.5,1,None,1,0.616798
...,...,...,...,...,...,...
2155,"(5, 7)",1.0,0.2,2500,0.0001,0.000000
2156,"(5, 7)",1.0,0.2,2500,0.001,0.000000
2157,"(5, 7)",1.0,0.2,2500,0.1,0.000000
2158,"(5, 7)",1.0,0.2,2500,1,0.000000


In [18]:
mnb_scoring_df = mnb_scoring_df.sort_values(by=["accuracy"], ascending=False).reset_index(drop = True)
mnb_scoring_df

,ngrams,max_df,min_df,max_features,alpha,accuracy
0,"(1, 1)",0.5,1,1800,0.1,0.847769
1,"(1, 1)",0.75,1,1800,0.1,0.847769
2,"(1, 1)",1.0,1,1800,0.1,0.847769
3,"(1, 1)",0.75,1,2500,0.1,0.845144
4,"(1, 7)",0.75,1,2500,1,0.845144
...,...,...,...,...,...,...
2155,"(3, 5)",1.0,0.1,None,1,0.000000
2156,"(3, 5)",1.0,0.1,None,10,0.000000
2157,"(3, 5)",1.0,0.1,1000,0.00001,0.000000
2158,"(3, 5)",1.0,0.1,1000,0.0001,0.000000


#### Вместе с Support Vector Machine (SVM)



<img src="https://scikit-learn.ru/wp-content/uploads/2021/10/image-74.png" />

In [19]:
from sklearn import svm

# Dataframe out of all combinations of the params
svm_scoring_df = pd.DataFrame(
    list(itertools.product(*textParamsValues, *tfidfParamsValues, *svmParamsValues)), 
    columns=[*textParamsKeys, *tfidfParamsKeys, *svmParamsKeys],
    dtype=object # Pandas converts None -> NaN otherwise
)
# svm_scoring_df.drop(svm_scoring_df[svm_scoring_df['min_df'] > svm_scoring_df['max_df']].index, inplace=True)
svm_scoring_df["accuracy"] = ""

svm_scoring_df

,ngrams,max_df,min_df,max_features,C,kernel,degree,gamma,accuracy
0,"(2, 2)",0.5,1,None,1.0,rbf,3,scale,
1,"(2, 2)",0.5,1,None,1.0,rbf,3,auto,
2,"(2, 2)",0.5,1,None,2.0,rbf,3,scale,
3,"(2, 2)",0.5,1,None,2.0,rbf,3,auto,
4,"(2, 2)",0.5,1,None,3.0,rbf,3,scale,
...,...,...,...,...,...,...,...,...,...
2875,"(5, 7)",1.0,0.2,2500,2.0,rbf,3,auto,
2876,"(5, 7)",1.0,0.2,2500,3.0,rbf,3,scale,
2877,"(5, 7)",1.0,0.2,2500,3.0,rbf,3,auto,
2878,"(5, 7)",1.0,0.2,2500,4.0,rbf,3,scale,


In [20]:
%%time
def score_svm(param_row):
    max_df, min_df, max_features = itemgetter('max_df', 'min_df', 'max_features')(param_row)
    vectorizer = TfidfVectorizer(analyzer=lambda x: x, max_df=max_df, min_df=min_df, max_features=max_features)

    ngrams = itemgetter('ngrams')(param_row)
    ngrammed_tokens = lemmatized_tokens.apply(lambda row: word_ngram(row, ngrams))
    
    try:
        points = vectorizer.fit_transform(ngrammed_tokens)
        values = sentiments['sentiment']
    except ValueError:
        # Handle bad params
        return 0.0
    
    x_train, x_test, y_train, y_test = train_test_split(points, values, test_size=0.25, random_state=42)

    C, kernel, degree, gamma = itemgetter('C', 'kernel', 'degree', 'gamma')(param_row)
    clf = svm.SVC(C=C, kernel=kernel, degree=degree, gamma=gamma)
    clf.fit(x_train, y_train)

    y_pred = clf.predict(x_test)

    accuracy = classification_report(y_test, y_pred, output_dict=True, zero_division=0)['accuracy']
    return accuracy

print(f"Estimated to take {len(svm_scoring_df)} fits or {round(len(svm_scoring_df) / 240)} minutes")

accuracies = Parallel(n_jobs=-1)(delayed(score_svm)(iterrow[1]) for iterrow in svm_scoring_df.iterrows())
svm_scoring_df['accuracy'] = pd.DataFrame(accuracies)
beep()

svm_scoring_df

Estimated to take 2880 fits or 12 minutes
CPU times: user 39.8 s, sys: 1.48 s, total: 41.3 s
Wall time: 14min 17s


,ngrams,max_df,min_df,max_features,C,kernel,degree,gamma,accuracy
0,"(2, 2)",0.5,1,None,1.0,rbf,3,scale,0.564304
1,"(2, 2)",0.5,1,None,1.0,rbf,3,auto,0.501312
2,"(2, 2)",0.5,1,None,2.0,rbf,3,scale,0.653543
3,"(2, 2)",0.5,1,None,2.0,rbf,3,auto,0.501312
4,"(2, 2)",0.5,1,None,3.0,rbf,3,scale,0.653543
...,...,...,...,...,...,...,...,...,...
2875,"(5, 7)",1.0,0.2,2500,2.0,rbf,3,auto,0.000000
2876,"(5, 7)",1.0,0.2,2500,3.0,rbf,3,scale,0.000000
2877,"(5, 7)",1.0,0.2,2500,3.0,rbf,3,auto,0.000000
2878,"(5, 7)",1.0,0.2,2500,4.0,rbf,3,scale,0.000000


In [21]:
svm_scoring_df = svm_scoring_df.sort_values(by=["accuracy"], ascending=False).reset_index(drop = True)
svm_scoring_df

,ngrams,max_df,min_df,max_features,C,kernel,degree,gamma,accuracy
0,"(1, 1)",0.75,1,None,1.0,rbf,3,scale,0.855643
1,"(1, 1)",0.5,1,None,1.0,rbf,3,scale,0.855643
2,"(1, 1)",1.0,1,None,1.0,rbf,3,scale,0.855643
3,"(1, 7)",1.0,1,1800,1.0,rbf,3,scale,0.850394
4,"(1, 7)",0.5,1,1800,1.0,rbf,3,scale,0.850394
...,...,...,...,...,...,...,...,...,...
2875,"(3, 5)",1.0,0.2,1800,4.0,rbf,3,auto,0.000000
2876,"(3, 5)",1.0,0.2,1800,4.0,rbf,3,scale,0.000000
2877,"(3, 5)",1.0,0.2,1800,3.0,rbf,3,auto,0.000000
2878,"(3, 5)",1.0,0.2,1800,3.0,rbf,3,scale,0.000000


#### Итог

Лучший классификатор, со следующими параметрами:

In [22]:
best_svm_params = svm_scoring_df.iloc[0]
best_svm_params

ngrams            (1, 1)
max_df              0.75
min_df                 1
max_features        None
C                    1.0
kernel               rbf
degree                 3
gamma              scale
accuracy        0.855643
Name: 0, dtype: object

In [215]:
# best_svm_params = pd.DataFrame({
#         "ngrams": [(1, 1)],
#         "max_df": [0.75],
#         "min_df": [1],
#         "max_features": [None],
#         "C": [1.0],
#         "kernel": ["rbf"],
#         "degree": [3],
#         "gamma": ["scale"],
#         "accuracy": [0.855643]
#     },
# ).iloc[0]

In [23]:
max_df, min_df, max_features = itemgetter('max_df', 'min_df', 'max_features')(best_svm_params)
vectorizer = TfidfVectorizer(analyzer=lambda x: x, max_df=max_df, min_df=min_df, max_features=max_features)

ngrams = itemgetter('ngrams')(best_svm_params)
ngrammed_tokens = lemmatized_tokens.apply(lambda row: word_ngram(row, ngrams))

points = vectorizer.fit_transform(ngrammed_tokens)
values = sentiments['sentiment']

x_train, x_test, y_train, y_test = train_test_split(points, values, test_size=0.25, random_state=42)

C, kernel, degree, gamma = itemgetter('C', 'kernel', 'degree', 'gamma')(best_svm_params)
best_clf = svm.SVC(C=C, kernel=kernel, degree=degree, gamma=gamma)
best_clf.fit(x_train, y_train)

SVC()

In [24]:
y_pred = best_clf.predict(x_test)

print(classification_report(y_test, y_pred, zero_division=0))

              precision    recall  f1-score   support

         joy       0.84      0.87      0.86       190
         sad       0.87      0.84      0.85       191

    accuracy                           0.86       381
   macro avg       0.86      0.86      0.86       381
weighted avg       0.86      0.86      0.86       381



## 6.&nbsp;Примените шаги предобработки на наборе данных COVID19 Tweets

[COVID19 Tweets](https://www.kaggle.com/datasets/gpreda/covid19-tweets)

In [25]:
import os
os.environ['KAGGLE_USERNAME'] = "whydoihavetodothiseh"
os.environ['KAGGLE_KEY'] = "1c0534cf33c5d00bb380dc4aebadd907"

!kaggle datasets download -d gpreda/covid19-tweets
!unzip -o $(ls -hat | grep -m 1 ".zip")

print()
!ls | grep ".csv"

covid19-tweets.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  covid19-tweets.zip
  inflating: covid19_tweets.csv      

covid19_tweets.csv
finalSentimentdata2.csv


In [26]:
import numpy as np
import pandas as pd

tweets = pd.read_csv("covid19_tweets.csv")
tweets.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False


In [27]:
lowercased_tweets = tweets.apply(lambda row: to_lower(row['text']), axis=1)
lowercased_tweets.head()

0    if i smelled the scent of hand sanitizers toda...
1    hey @yankees @yankeespr and @mlb - wouldn't it...
2    @diane3443 @wdunlap @realdonaldtrump trump nev...
3    @brookbanktv the one gift #covid19 has give me...
4    25 july : media bulletin on novel #coronavirus...
dtype: object

In [28]:
filtered_tweets = lowercased_tweets.apply(lambda row: filter_out_meta(row))
filtered_tweets.head()

0    if i smelled the scent of hand sanitizers toda...
1    hey and wouldn t it have made more sense to ha...
2    trump never once claimed was a hoax we all cla...
3    the one gift has give me is an appreciation fo...
4                      25 july media bulletin on novel
dtype: object

In [29]:
tweet_tokens = filtered_tweets.apply(lambda row: word_tokenize(row))
tweet_tokens.head()

0    [if, i, smelled, the, scent, of, hand, sanitiz...
1    [hey, and, wouldn, t, it, have, made, more, se...
2    [trump, never, once, claimed, was, a, hoax, we...
3    [the, one, gift, has, give, me, is, an, apprec...
4               [25, july, media, bulletin, on, novel]
dtype: object

In [30]:
cleaned_tweet_tokens = tweet_tokens.apply(lambda row: [token for token in row if token not in stop_words and token not in punctuation])
cleaned_tweet_tokens.head()

0    [smelled, scent, hand, sanitizers, today, some...
1           [hey, made, sense, players, pay, respects]
2         [trump, never, claimed, hoax, claim, effort]
3    [one, gift, give, appreciation, simple, things...
4                   [25, july, media, bulletin, novel]
dtype: object

In [31]:
lemmatized_tweet_tokens = cleaned_tweet_tokens.apply(lambda row: list(map(lemmatizer.lemmatize, row)))
lemmatized_tweet_tokens.head()

0    [smelled, scent, hand, sanitizers, today, some...
1             [hey, made, sense, player, pay, respect]
2         [trump, never, claimed, hoax, claim, effort]
3    [one, gift, give, appreciation, simple, thing,...
4                  [25, july, medium, bulletin, novel]
dtype: object

## 7.&nbsp;С помощью обученного классификатора найдите посты из набора данных COVID19 Tweets, которые можно отнести к классу sad. 
Какой процент от общего количества постов они составляют?

In [34]:
max_df, min_df, max_features = itemgetter('max_df', 'min_df', 'max_features')(best_svm_params)
max_features = best_clf.n_features_in_
vectorizer = TfidfVectorizer(analyzer=lambda x: x, max_features=max_features)

ngrams = itemgetter('ngrams')(best_svm_params)
ngrammed_tweet_tokens = lemmatized_tweet_tokens.apply(lambda row: word_ngram(row, ngrams))

tweet_points = vectorizer.fit_transform(ngrammed_tweet_tokens)
tweet_values = best_clf.predict(tweet_points)

tweet_values

array(['joy', 'joy', 'joy', ..., 'joy', 'joy', 'joy'], dtype=object)

In [35]:
tweets_pred_df = pd.DataFrame({"text": tweets['text'], "sentiment": tweet_values})
tweets_pred_df

,text,sentiment
0,If I smelled the scent of hand sanitizers toda...,joy
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,joy
2,@diane3443 @wdunlap @realDonaldTrump Trump nev...,joy
3,@brookbanktv The one gift #COVID19 has give me...,joy
4,25 July : Media Bulletin on Novel #CoronaVirus...,joy
...,...,...
179103,Thanks @IamOhmai for nominating me for the @WH...,joy
179104,2020! The year of insanity! Lol! #COVID19 http...,joy
179105,@CTVNews A powerful painting by Juan Lucena. I...,joy
179106,"More than 1,200 students test positive for #CO...",joy


### Грустные твиты

In [36]:
sad_tweets = tweets_pred_df[tweets_pred_df["sentiment"] == "sad"]
sad_tweets

,text,sentiment
5,#coronavirus #covid19 deaths continue to rise....,sad
10,49K+ Covid19 cases still no response from \n@c...,sad
17,Second wave of #COVID19 in Flanders..back to m...,sad
18,It is during our darkest moments that we must ...,sad
19,COVID Update: The infection rate in Florida is...,sad
...,...,...
179097,@politvidchannel Just not the lives of #COVID1...,sad
179098,Report #COVID19 outbreaks in K-12 schools here...,sad
179099,I have NOTHING BUT 💚 for the @NBA these days.....,sad
179100,Wallkill school nurse adds COVID-19 monitoring...,sad


Доля грустных твитов

In [37]:
sad_tweets_share = str(round((len(sad_tweets) / len(tweets_pred_df)) * 100)) + "%"

print("Sad tweets share:", sad_tweets_share)

Sad tweets share: 29%


## 8.&nbsp;Проанализируйте тональность постов из набора данных COVID19 Tweets с помощью библиотеки TextBlob (Dostoevsky*)

Какое процентное соотношение постов разных настроений наблюдается?

[TextBlob 0.16.0 documentation | readthedocs.io](https://textblob.readthedocs.io/en/dev/advanced_usage.html#sentiment-analyzers)

[Dostoevsky | github.com](https://github.com/bureaucratic-labs/dostoevsky)

In [38]:
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

nltk.download('movie_reviews')

nb_clf = NaiveBayesAnalyzer()

def get_textblob_sentiment(text):
    blob = TextBlob(text, analyzer=nb_clf)
    return blob.sentiment.classification

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [39]:
# Undoing tokenization
lemmatized_tweets = lemmatized_tweet_tokens.apply(lambda row: ' '.join(row))
lemmatized_tweets.head()

0    smelled scent hand sanitizers today someone pa...
1                    hey made sense player pay respect
2                trump never claimed hoax claim effort
3    one gift give appreciation simple thing always...
4                        25 july medium bulletin novel
dtype: object

In [40]:
a_tweet = lemmatized_tweets[179106]
blob = TextBlob(a_tweet, analyzer=nb_clf)

print(a_tweet)
print(blob.tokens, "\n")
print(blob.sentiment)

1 200 student test positive major university abc news
['1', '200', 'student', 'test', 'positive', 'major', 'university', 'abc', 'news'] 

Sentiment(classification='pos', p_pos=0.5770351267237228, p_neg=0.42296487327627846)


In [41]:
textblob_tweet_sentiments = lemmatized_tweets.apply(get_textblob_sentiment)
textblob_tweet_sentiments

0         pos
1         neg
2         pos
3         pos
4         pos
         ... 
179103    pos
179104    neg
179105    pos
179106    pos
179107    pos
Length: 179108, dtype: object

In [42]:
tweets_pred_df['textblob sentiment'] = textblob_tweet_sentiments
print("Textblob sentiments:", tweets_pred_df['textblob sentiment'].unique())

tweets_pred_df

Textblob sentiments: ['pos' 'neg']


,text,sentiment,textblob sentiment
0,If I smelled the scent of hand sanitizers toda...,joy,pos
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,joy,neg
2,@diane3443 @wdunlap @realDonaldTrump Trump nev...,joy,pos
3,@brookbanktv The one gift #COVID19 has give me...,joy,pos
4,25 July : Media Bulletin on Novel #CoronaVirus...,joy,pos
...,...,...,...
179103,Thanks @IamOhmai for nominating me for the @WH...,joy,pos
179104,2020! The year of insanity! Lol! #COVID19 http...,joy,neg
179105,@CTVNews A powerful painting by Juan Lucena. I...,joy,pos
179106,"More than 1,200 students test positive for #CO...",joy,pos


### Негативные твиты

In [45]:
neg_tweets = tweets_pred_df[tweets_pred_df["textblob sentiment"] == "neg"]
neg_tweets

,text,sentiment,textblob sentiment
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,joy,neg
7,You now have to wear face coverings when out s...,joy,neg
9,POPE AS GOD - Prophet Sadhu Sundar Selvaraj. W...,joy,neg
10,49K+ Covid19 cases still no response from \n@c...,sad,neg
13,Let's all protect ourselves from #COVID19.\nIt...,joy,neg
...,...,...,...
179092,@politvidchannel The Trump Administration's In...,joy,neg
179094,"""Symptom screening fails to identify most #Cov...",joy,neg
179095,#COVID19 Update: 23 new cases today in the Tri...,joy,neg
179099,I have NOTHING BUT 💚 for the @NBA these days.....,sad,neg


In [46]:
neg_tweets_share = str(round((len(neg_tweets) / len(tweets_pred_df)) * 100)) + "%"

print("Negative tweets share:", neg_tweets_share)

Negative tweets share: 29%


### Грустно-негативные твиты

In [47]:
sad_neg_tweets = tweets_pred_df[(tweets_pred_df["textblob sentiment"] == "neg") & (tweets_pred_df["sentiment"] == "sad")]
sad_neg_tweets

,text,sentiment,textblob sentiment
10,49K+ Covid19 cases still no response from \n@c...,sad,neg
42,Volume for those at the back please. 🔊 #COVID1...,sad,neg
43,"#Delhi reports 1,142 #COVID19 positive cases a...",sad,neg
61,I miss isopropyl alcohol so much!!!! Ethanol i...,sad,neg
69,"While you at it, please follow me\n\nhttps://t...",sad,neg
...,...,...,...
179056,@ProjectLincoln it looks like @realDonaldTrump...,sad,neg
179066,Standard &amp; Poor’s affirms Georgia at ‘BB’ ...,sad,neg
179068,Our latest paper on estimating the risk of #CO...,sad,neg
179076,You get a discount because of the millions of ...,sad,neg


In [50]:
sad_neg_tweets_share = str(round((len(sad_neg_tweets) / len(tweets_pred_df)) * 100)) + "%"

print("Both Sad and Negative tweets share:", sad_neg_tweets_share)

Both Sad and Negative tweets share: 8%
